<a href="https://colab.research.google.com/github/ridhodwidharmawan/Landsat-Classification-Using-Neural-Network/blob/ipynb/Builtup_ANN_Landsat8_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Install Pyrsgis
!pip install pyrsgis

In [3]:
#Get dataset
!git clone https://github.com/ridhodwidharmawan/Landsat-Classification-Using-Neural-Network

Cloning into 'Landsat-Classification-Using-Neural-Network'...
remote: Enumerating objects: 54, done.
remote: Total 54 (delta 0), reused 0 (delta 0), pack-reused 54
Unpacking objects: 100% (54/54), done.


In [4]:
import os
import numpy as np
from tensorflow import keras
from pyrsgis import raster
from pyrsgis.convert import changeDimension
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score

Warning! matplotlib_scalebar library not found.


In [5]:
# Change the directory
os.chdir("Landsat-Classification-Using-Neural-Network/dataset")

In [6]:
# Assign file names
featuredata_inp = 'LC08_120065_20180505_featuresdata.tif'
labeldata_inp = 'builtup_labeldata.tif'
unlabeleddata_inp = 'LC08_120065_20190625_unlabeleddata.tif'

In [7]:
# Read the rasters as array
ds1, featuredata = raster.read(featuredata_inp, bands='all')
ds2, labeldata = raster.read(labeldata_inp, bands=1)
ds3, unlabeleddata = raster.read(unlabeleddata_inp, bands='all')

In [8]:
# Print the size of the arrays
print("Feature data multiband image shape: ", featuredata.shape)
print("Label data builtup area image shape: ", labeldata.shape)
print("Unlabeled data multiband image shape: ", unlabeleddata.shape)

Feature data multiband image shape:  (6, 844, 843)
Label data builtup area image shape:  (844, 843)
Unlabeled data multiband image shape:  (6, 3798, 3122)


In [9]:
# Clean the labelled data to replace NoData values by zero
labeldata = (labeldata == 1).astype(int)

In [10]:
# Reshape the array to single dimensional array
featuredata = changeDimension(featuredata)
labeldata = changeDimension (labeldata)
unlabeleddata = changeDimension(unlabeleddata)
nBands = featuredata.shape[1]

In [11]:
# Print new dimension size
print("Feature data multiband image shape: ", featuredata.shape)
print("Label data builtup area image shape: ", labeldata.shape)
print("Unlabeled data multiband image shape: ", unlabeleddata.shape)

Feature data multiband image shape:  (711492, 6)
Label data builtup area image shape:  (711492,)
Unlabeled data multiband image shape:  (11857356, 6)


In [12]:
# Split testing and training datasets
xTrain, xTest, yTrain, yTest = train_test_split(featuredata, labeldata, test_size=0.4, random_state=42)

In [13]:
# Print train size
print(xTrain.shape)
print(yTrain.shape)

(426895, 6)
(426895,)


In [14]:
# Print test size
print(xTest.shape)
print(yTest.shape)

(284597, 6)
(284597,)


In [15]:
# Normalise the data
xTrain = xTrain / 10000
xTest = xTest / 10000
unlabeleddata = unlabeleddata / 10000

In [16]:
# Reshape the data
xTrain = xTrain.reshape((xTrain.shape[0], 1, xTrain.shape[1]))
xTest = xTest.reshape((xTest.shape[0], 1, xTest.shape[1]))
unlabeleddata = unlabeleddata.reshape((unlabeleddata.shape[0], 1, unlabeleddata.shape[1]))

In [17]:
# Print the shape of reshaped data
print(xTrain.shape, xTest.shape, unlabeleddata.shape)

(426895, 1, 6) (284597, 1, 6) (11857356, 1, 6)


In [18]:
# Define the parameters of the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, nBands)),
    keras.layers.Dense(14, activation='relu'),
    keras.layers.Dense(2, activation='softmax')])

In [19]:
# Define the accuracy metrics and parameters
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [20]:
# Run the model
model.fit(xTrain, yTrain, epochs=5)

Epoch 1/5
13341/13341 [==============================] - 14s 1ms/step - loss: 0.4678 - accuracy: 0.7884
Epoch 2/5
13341/13341 [==============================] - 14s 1ms/step - loss: 0.4226 - accuracy: 0.8185
Epoch 3/5
13341/13341 [==============================] - 13s 1ms/step - loss: 0.4136 - accuracy: 0.8248
Epoch 4/5
13341/13341 [==============================] - 13s 999us/step - loss: 0.4102 - accuracy: 0.8267
Epoch 5/5
13341/13341 [==============================] - 13s 998us/step - loss: 0.4084 - accuracy: 0.8275


In [21]:
# Predict for test data 
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]

In [22]:
# Calculate and display the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

In [23]:
print("Confusion matrix: for 14 nodes\n", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

Confusion matrix: for 14 nodes
 [[144030  20553]
 [ 28360  91654]]

P-Score: 0.817, R-Score: 0.764


In [24]:
predicted = model.predict(unlabeleddata)
predicted = predicted[:,1]

In [25]:
#Export raster
prediction = np.reshape(predicted, (ds3.RasterYSize, ds3.RasterXSize))
outFile = 'builtup_predicted.tif'
raster.export(prediction, ds3, filename=outFile, dtype='float')